# Middleware

## Agent Loop

<div align="center">
  <img src="./images/react_agent.png" style="width: 40%; max-width: 400px;" alt="Agent Loop">
</div>

## Old Langgraph Hooks

<div align="center">
  <img src="./images/old_langgraph_hooks.png" style="width: 40%; max-width: 400px;" alt="Old Langgraph Hooks">
</div>

## Middleware Hooks

<div align="center">
  <img src="./images/new_middleware_hooks.png" style="width: 40%; max-width: 400px;" alt="Middleware Hooks">
</div>

```python
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[...],
    middleware=[
        ...
    ],
)
```

In [ ]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from tools import lookup_stock_symbol, fetch_stock_data_raw, place_order
from helpers import draw_mermaid_png

system_prompt = """
You are a financial advisor assistant. Use the provided tools to ground your answers
in up-to-date market data. Be concise, factual, and risk-aware.

Be decisive: when you have sufficient information to act, proceed with tool calls without
asking for confirmation. Only if information is missing or uncertain, ask a concise 
clarifying question.

When preparing or describing actions, include appropriate parameters (e.g., symbol, shares,
limit price, budgets) based on available data. Do not fabricate numbers or facts.
"""

agent = create_agent(
    model="gpt-4o-mini",
    tools=[lookup_stock_symbol, fetch_stock_data_raw, place_order],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),

    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "place_order": { "allowed_decisions": ["approve", "edit", "reject"]}
            }
        ),
    ],
)


draw_mermaid_png(agent)

## testing

In [ ]:
from langchain.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

response = agent.invoke({
    "messages": [
        HumanMessage("Buy a Tesla stock at the current price.")
    ]
}, config=config)
for message in response['messages']:
    message.pretty_print()

In [ ]:
"__interrupt__" in response

In [ ]:
interrupts = response["__interrupt__"]
interrupts[0].value

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]}  # or "edit", "reject"
    ), config=config
)
for message in response['messages']:
    message.pretty_print()

## Run in Studio and Test in Chat UI

From `studio` directory run:
```
langgraph dev
```

Navigate to [https://agentchat.vercel.app/](https://agentchat.vercel.app/) in your browser to access the LangGraph Chat UI.